In [ ]:
import cvxpy as cp
import numpy as np
from scipy.stats import norm
from numpy.linalg import norm as np_norm

# Assuming necessary data is loaded here
# For demonstration, generating synthetic data
Demand = np.random.rand(1000)  # Placeholder for actual demand data
Features = np.random.rand(1000, 3)  # Placeholder for actual features data

# Parameters
lntr = 12 * 7 * 16  # Training data size
lnva = lntr // 2  # Validation data size
delay = 3
p = 12 * 0  # Adjust as necessary
b = 2.5 / 3.5
h = 1 / 3.5
r = b / (b + h)

# Placeholder for OLS regression results
Qfac = np.zeros((lnva, 1 + Features.shape[1]))
QfacD = np.zeros(lnva)
muD = np.zeros(lnva)
sigmaD = np.zeros(lnva)

for t in range(lntr + 1, lntr + lnva + 1):
    # Normalize Features
    FeaturesT = Features[t - lntr:t, :] / np_norm(Features[t - lntr:t, :], np.inf, axis=0)
    
    # Estimate coefficients via OLS
    beta0 = cp.Variable()
    beta1 = cp.Variable(Features.shape[1])
    Res = cp.Variable(lntr)
    objective = cp.Minimize(cp.sum_squares(Res))
    constraints = [Res == Demand[t - lntr - 1 + i + delay] - beta0 - cp.matmul(beta1, FeaturesT[i, :].T) for i in range(lntr)]
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # Estimate sigma coefficients via OLS (logarithm of squared residuals)
    delta0 = cp.Variable()
    delta1 = cp.Variable(Features.shape[1])
    Res2 = cp.Variable(lntr)
    objective2 = cp.Minimize(cp.sum_squares(Res2))
    constraints2 = [Res2 == cp.log(cp.square(Res[i])) - delta0 - cp.matmul(delta1, FeaturesT[i, :].T) for i in range(lntr)]
    prob2 = cp.Problem(objective2, constraints2)
    prob2.solve()

    # Calculate optimal order quantity based on estimates
    current_features = Features[t, :] / np_norm(Features[t - lntr:t, :], np.inf)
    muD[t - lntr] = beta0.value + beta1.value.T @ current_features
    sigmaD[t - lntr] = np.exp((delta0.value + delta1.value.T @ current_features) / 2)
    QfacD[t - lntr] = muD[t - lntr] + sigmaD[t - lntr] * norm.ppf(r)

# Assuming `nvcost` function is defined elsewhere and computes the cost
Valfac = [nvcost(QfacD[i], Demand[i + lntr + delay], b, h) for i in range(lnva)]

# Save the results (Adjust according to your data structure and requirement)
